In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from scipy.ndimage import label
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import json
from collections import defaultdict
from matplotlib.backends.backend_pdf import PdfPages

MAX_STEPS = 10

def generate_percolation_lattice(size, p):
    return np.random.choice([0, 1], (size, size), p=[1-p, p]).astype(np.uint8)

def check_percolation(lattice):
    labeled, _ = label(lattice)
    top = set(labeled[0]) - {0}
    bottom = set(labeled[-1]) - {0}
    left = set(labeled[:,0]) - {0}
    right = set(labeled[:,-1]) - {0}
    return float(bool(top & bottom) or bool(left & right))

def first_coarse_graining(binary_lattice, dim):
    """Average non-overlapping dim×dim blocks."""
    t = torch.tensor(binary_lattice, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    patches = F.unfold(t, kernel_size=dim, stride=dim)
    patches = patches.permute(0, 2, 1)
    coarse_vals = patches.mean(dim=2)
    H, W = binary_lattice.shape
    new_h, new_w = H // dim, W // dim
    return coarse_vals.view(1, 1, new_h, new_w).squeeze(0)

class PercolationModel(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
        self.rule = nn.Sequential(
            nn.Linear(dim * dim, 32),  # First layer with 32 neurons
            nn.ReLU(),
            nn.Linear(32, 32),        # Second layer with 32 neurons
            nn.ReLU(),
            nn.Linear(32, 1),          # Final output layer
            nn.Sigmoid()
        )
    
    def forward(self, x, max_steps=MAX_STEPS):
        b, c, H, W = x.shape
        for _ in range(max_steps):
            if H < self.dim or W < self.dim:
                break
            patches = F.unfold(x, kernel_size=self.dim, stride=self.dim)
            patches = patches.permute(0, 2, 1).contiguous()
            out = self.rule(patches.view(-1, self.dim*self.dim))
            new_h, new_w = H // self.dim, W // self.dim
            x = out.view(b, 1, new_h, new_w)
            _, _, H, W = x.shape
        return x.squeeze(1).view(b, -1)

def prepare_dataset(N, sizes):
    data = []
    for _ in tqdm(range(int(N/2)), desc="Generating data"):
        p = np.random.uniform(0.1, 0.9)
        size = np.random.choice(sizes)
        L = generate_percolation_lattice(size, p)
        data.append((L, check_percolation(L)))
    for _ in tqdm(range(int(N/2)), desc="Generating critical data"):
        p = np.random.uniform(0.55, 0.65)
        size = np.random.choice(sizes)
        L = generate_percolation_lattice(size, p)
        data.append((L, check_percolation(L)))
    return data

def train_epoch(model, device, data, batch_size, opt, crit, dim):
    model.train()
    total_loss = 0.0
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        processed = []
        for x, y in batch:
            cg_lattice = first_coarse_graining(x, dim)
            h, w = cg_lattice.shape[-2], cg_lattice.shape[-1]
            processed.append((cg_lattice, y, (h, w)))
        
        groups = {}
        for cg, y, size in processed:
            if size not in groups:
                groups[size] = []
            groups[size].append((cg, y))
        
        group_loss = 0.0
        for size_key, group in groups.items():
            lattices = [item[0] for item in group]
            labels = [item[1] for item in group]
            inputs = torch.stack(lattices).to(device)
            targets = torch.tensor(labels, dtype=torch.float32, device=device)
            
            opt.zero_grad()
            outputs = model(inputs)
            loss = crit(outputs.view(-1), targets)
            loss.backward()
            opt.step()
            group_loss += loss.item() * len(group)
        
        total_loss += group_loss
    
    return total_loss / len(data)

def test_systems(model, dim, power, device='cpu',
                 num_tests=50, system_size='standard',
                 p_range=(0,1), verbose=True):
    model.to(device).eval()
    mapping = {
        f'{dim}^2': power - 1,
        f'{dim}^3': power,
        f'{dim}^4': power + 1,
        f'{dim}^5': power + 2,
        f'{dim}^6': power + 3,
        f'{dim}^7': power + 4
    }
    size_power = mapping[system_size]
    L = dim ** size_power

    results = []
    for _ in range(num_tests):
        p = np.random.uniform(*p_range)
        raw = generate_percolation_lattice(L, p)
        lbl = check_percolation(raw)
        coarse = first_coarse_graining(raw, dim)
        inp = coarse.unsqueeze(0).to(device)
        
        with torch.no_grad():
            out = model(inp).view(-1).item()
        results.append((raw, lbl, out))

    acc = sum((pred > 0.5) == lbl for _, lbl, pred in results) / num_tests
    pos = [pred for _, lbl, pred in results if lbl == 1]
    neg = [pred for _, lbl, pred in results if lbl == 0]
    
    metrics = {
        'accuracy': acc,
        'avg_pred_perc': np.mean(pos) if pos else 0,
        'avg_pred_non_perc': np.mean(neg) if neg else 0
    }
    
    if verbose:
        print(f"\nAfter manual first coarse-grain -> NN cascade on {L}×{L}:")
        print(f" Accuracy        : {acc:.2%}")
        print(f" Avg pred | Perc     : {metrics['avg_pred_perc']:.3f}")
        print(f" Avg pred | Non-Perc : {metrics['avg_pred_non_perc']:.3f}")

    return metrics

def visualize_rule(model, dim, device='cpu'):
    p_values = np.linspace(0, 1, 100)
    model.eval()
    mean_outputs = []
    with torch.no_grad():
        for p in p_values:
            # Create a single input vector with all values = p
            inputs = torch.full((1, dim*dim), p, 
                               dtype=torch.float32, device=device)
            # Get deterministic output
            output = model.rule(inputs).item()
            mean_outputs.append(output)
    
    mean_outputs = np.array(mean_outputs)
    diff = mean_outputs - p_values
    crossings = []
    for i in range(len(p_values) - 1):
        if p_values[i] < 0.1 or p_values[i] > 0.9:
            continue
        if diff[i] * diff[i+1] <= 0:
            x1, x2 = p_values[i], p_values[i+1]
            y1, y2 = diff[i], diff[i+1]
            if y1 == y2:
                continue
            cross = x1 - y1 * (x2 - x1) / (y2 - y1)
            crossings.append(cross)
    
    valid_mask = (p_values >= 0.1) & (p_values <= 0.9)
    valid_p = p_values[valid_mask]
    valid_out = mean_outputs[valid_mask]
    abs_diff = np.abs(valid_out - valid_p)
    if len(valid_p) > 0:
        p_c_model = valid_p[np.argmin(abs_diff)]
    else:
        p_c_model = p_values[np.argmin(np.abs(mean_outputs - p_values))]
    
    idx = np.argmin(np.abs(p_values - p_c_model))
    
    if idx == 0:
        deriv = (mean_outputs[1] - mean_outputs[0]) / (p_values[1] - p_values[0])
    elif idx == len(p_values) - 1:
        deriv = (mean_outputs[-1] - mean_outputs[-2]) / (p_values[-1] - p_values[-2])
    else:
        deriv = (mean_outputs[idx+1] - mean_outputs[idx-1]) / (p_values[idx+1] - p_values[idx-1])
    
    lambda_val = abs(deriv)
    if lambda_val > 1:
        nu = np.log(dim) / np.log(lambda_val)
    else:
        nu = float('nan')
    
    return p_values, mean_outputs, p_c_model, crossings, deriv, nu

def run_experiment(dim, power, num_runs, device, ratio, epochs):
    all_rule_curves = []
    all_test_results = defaultdict(list)
    all_pc_values = []
    all_nu_values = []
    
    # Calculate model parameters to determine training samples
    model = PercolationModel(dim).to(device)
    num_params = sum(p.numel() for p in model.parameters())
    train_samples = int(num_params / ratio)
    
    print(f"\nStarting experiment for DIM={dim}")
    print(f"Parameters: {num_params}, Ratio: {ratio}, Training samples: {train_samples}")
    
    for run_idx in range(num_runs):
        print(f"\n{'='*40}")
        print(f"Run {run_idx+1}/{num_runs} for DIM={dim}, Ratio={ratio}")
        print(f"{'='*40}")
        
        sizes = [dim**2, dim**3, dim**4]
        train_data = prepare_dataset(train_samples, sizes)
        
        model = PercolationModel(dim).to(device)
        opt = optim.Adam(model.parameters(), lr=1e-3)
        crit = nn.BCELoss()
        
        # Training loop with custom epochs
        for epoch in range(1, epochs + 1):
            loss = train_epoch(model, device, train_data, 10, opt, crit, dim)
            if epoch % 50 == 0 or epoch == 1 or epoch == epochs:
                print(f"Epoch {epoch}/{epochs} — Loss: {loss:.4f}")
        
        # Test configurations - adjust sample counts
        test_configs = [
            {'system_size': f'{dim}^2', 'num_tests': 100, 'p_range': (0.55, 0.65)},
            {'system_size': f'{dim}^3', 'num_tests': 100, 'p_range': (0.55, 0.65)},
            {'system_size': f'{dim}^4', 'num_tests': 100, 'p_range': (0.55, 0.65)},
            {'system_size': f'{dim}^5', 'num_tests': 10, 'p_range': (0.55, 0.65)},
        ]
        
        for config in test_configs:
            key = f"{config['system_size']}_{config['p_range'][0]}-{config['p_range'][1]}"
            metrics = test_systems(
                model, dim, power, device,
                num_tests=config['num_tests'],
                system_size=config['system_size'],
                p_range=config['p_range'],
                verbose=False
            )
            all_test_results[key].append(metrics)
        
        p_vals, outputs, p_c_model, crossings, deriv, nu = visualize_rule(model, dim, device)
        all_rule_curves.append((p_vals, outputs, p_c_model, crossings))
        all_pc_values.append(p_c_model)
        all_nu_values.append(nu)
        
        print(f"Critical point estimate: {p_c_model:.4f}")
        print(f"Critical exponent ν: {nu:.4f}")
        
        del model, opt, crit, train_data
        torch.cuda.empty_cache()
    
    return all_rule_curves, all_test_results, all_pc_values, all_nu_values

def save_consolidated_results_for_ratio(dim, ratio, results):
    all_rule_curves, all_test_results, all_pc_values, all_nu_values = results
    
    # Create directory structure
    base_dir = f"results_dim_{dim}"
    ratio_dir = os.path.join(base_dir, f"ratio_{ratio}")
    os.makedirs(ratio_dir, exist_ok=True)
    
    # Save PDF report
    pdf_path = os.path.join(ratio_dir, "consolidated_results.pdf")
    with PdfPages(pdf_path) as pdf:
        valid_nu = [nu for nu in all_nu_values if not np.isnan(nu)]
        avg_pc = np.mean(all_pc_values)
        std_pc = np.std(all_pc_values)
        avg_nu = np.mean(valid_nu) if valid_nu else float('nan')
        std_nu = np.std(valid_nu) if valid_nu else float('nan')
        
        plt.figure(figsize=(10, 6))
        for i, (p_vals, outputs, p_c_model, crossings) in enumerate(all_rule_curves):
            plt.plot(p_vals, outputs, alpha=0.5, color='blue')
            valid_crossings = [cross for cross in crossings if 0.1 <= cross <= 0.9]
            for cross in valid_crossings:
                plt.scatter(cross, cross, color='black', s=30, zorder=3)
        
        plt.plot([0, 1], [0, 1], 'k--', label=r'$f(p) = p$', linewidth=1.5)
        plt.plot([], [], ' ', label=f'$p_c = {avg_pc:.3f} \\pm {std_pc:.3f}$')
        if not np.isnan(avg_nu):
            plt.plot([], [], ' ', label=f'$\\nu = {avg_nu:.3f} \\pm {std_nu:.3f}$')
        
        plt.xlabel('$p$', fontsize=16)
        plt.ylabel(r'$f_{\theta}(p\mathbf{1})$', fontsize=16)
        plt.title(f"Rule Projection - DIM={dim}, Ratio={ratio}", fontsize=20)
        plt.legend(fontsize=13)
        plt.xlim(0, 1)
        plt.ylim(0, 1)
        pdf.savefig(bbox_inches='tight')
        plt.close()
    
    # Save text report
    txt_path = os.path.join(ratio_dir, "consolidated_results.txt")
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(f"{'='*40}\n")
        f.write(f"Critical Point Analysis (DIM={dim}, Ratio={ratio})\n")
        f.write(f"{'='*40}\n\n")
        f.write("p_c values from each run:\n")
        for i, pc in enumerate(all_pc_values):
            f.write(f"Run {i+1}: {pc:.6f}\n")
        
        f.write(f"\nAverage p_c: {np.mean(all_pc_values):.6f}\n")
        f.write(f"Standard deviation: {np.std(all_pc_values):.6f}\n\n")
        
        f.write(f"Critical Exponent ν values:\n")
        for i, nu in enumerate(all_nu_values):
            status = "VALID" if not np.isnan(nu) else "INVALID (λ≤1)"
            f.write(f"Run {i+1}: {nu:.6f} [{status}]\n")
        
        if valid_nu:
            f.write(f"\nAverage ν (valid runs): {np.mean(valid_nu):.6f}\n")
            f.write(f"Standard deviation: {np.std(valid_nu):.6f}\n")
        else:
            f.write("\nNo valid ν values (all runs had λ≤1)\n")
        f.write("\n")
        
        f.write(f"{'='*40}\n")
        f.write(f"Test Performance Metrics (DIM={dim}, Ratio={ratio})\n")
        f.write(f"{'='*40}\n\n")
        
        for config, results_list in all_test_results.items():
            parts = config.split('_')
            system_size = parts[0]
            p_range = parts[1]
            
            accuracies = [r['accuracy'] for r in results_list]
            avg_perc = [r['avg_pred_perc'] for r in results_list]
            avg_non_perc = [r['avg_pred_non_perc'] for r in results_list]
            
            f.write(f"System: {system_size}, p-range: {p_range}\n")
            f.write("-"*50 + "\n")
            
            f.write("Run | Accuracy | Avg Perc | Avg Non-Perc\n")
            f.write("----|----------|----------|------------\n")
            for i in range(len(results_list)):
                f.write(f"{i+1:3d} | {accuracies[i]:.4f} | {avg_perc[i]:.4f} | {avg_non_perc[i]:.4f}\n")
            
            f.write("\nSummary Statistics:\n")
            f.write(f"Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}\n")
            f.write(f"Avg Perc: {np.mean(avg_perc):.4f} ± {np.std(avg_perc):.4f}\n")
            f.write(f"Avg Non-Perc: {np.mean(avg_non_perc):.4f} ± {np.std(avg_non_perc):.4f}\n")
            f.write("="*50 + "\n\n")
        f.write("\n\n")

def main():
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    POWER = 3
    
    # Define experiment parameters: {dim: {ratio: epochs}}
    EXPERIMENT_PARAMS = {
        3: {
            0.2: 10,
            0.5: 250,
            2: 110,
            5: 200
        },
        4: {
            0.2: 125,
            0.5: 40,
            2: 125,
            5: 250
        },
        5: {
            0.2: 25,
            0.5: 40,
            2: 250,
            5: 60
        }
    }
    
    # Run experiments for each dimension and ratio
    for dim, ratio_epochs in EXPERIMENT_PARAMS.items():
        print(f"\n\n{'='*50}")
        print(f"STARTING EXPERIMENTS FOR DIM={dim}")
        print(f"{'='*50}")
        
        for ratio, epochs in ratio_epochs.items():
            print(f"\n{'='*50}")
            print(f"STARTING RUNS FOR DIM={dim}, RATIO={ratio}")
            print(f"{'='*50}")
            
            results = run_experiment(
                dim=dim,
                power=POWER,
                num_runs=10,
                device=DEVICE,
                ratio=ratio,
                epochs=epochs
            )
            
            save_consolidated_results_for_ratio(dim, ratio, results)

if __name__ == "__main__":
    main()



STARTING EXPERIMENTS FOR DIM=3

STARTING RUNS FOR DIM=3, RATIO=0.2

Starting experiment for DIM=3
Parameters: 1409, Ratio: 0.2, Training samples: 7045

Run 1/10 for DIM=3, Ratio=0.2


Generating critical data: 100%|███████████████████████████████████████████████████████████████| 3522/3522 [00:00<00:00, 3600.92it/s]


Epoch 1/10 — Loss: 0.4088
Epoch 10/10 — Loss: 0.3005
Critical point estimate: 0.5758
Critical exponent ν: 0.7907

Run 2/10 for DIM=3, Ratio=0.2


Generating critical data: 100%|███████████████████████████████████████████████████████████████| 3522/3522 [00:01<00:00, 2715.77it/s]


Epoch 1/10 — Loss: 0.4032
Epoch 10/10 — Loss: 0.2972
Critical point estimate: 0.5657
Critical exponent ν: 0.7940

Run 3/10 for DIM=3, Ratio=0.2


...

Epoch 1/250 — Loss: 0.6726
Epoch 50/250 — Loss: 0.2807
Epoch 100/250 — Loss: 0.2774
Epoch 150/250 — Loss: 0.2173
Epoch 200/250 — Loss: 0.1924
Epoch 250/250 — Loss: 0.1781
Critical point estimate: 0.5657
Critical exponent ν: 0.7239

Run 6/10 for DIM=5, Ratio=2


Generating critical data: 100%|██████████████████████████████████████████████████████████████████| 480/480 [00:04<00:00, 112.73it/s]


Epoch 1/250 — Loss: 0.6778
Epoch 50/250 — Loss: 0.3098
Epoch 100/250 — Loss: 0.4598
Epoch 150/250 — Loss: 0.2089
Epoch 200/250 — Loss: 0.2091
Epoch 250/250 — Loss: 0.2326
Critical point estimate: 0.5859
Critical exponent ν: 0.9250

Run 7/10 for DIM=5, Ratio=2


Generating critical data: 100%|██████████████████████████████████████████████████████████████████| 480/480 [00:03<00:00, 139.82it/s]


Epoch 1/250 — Loss: 0.6585
Epoch 50/250 — Loss: 0.2168
